In [218]:
import json
import os
import pickle
import time
from contextlib import contextmanager
from typing import List, NoReturn, Optional, Tuple, Union

import faiss
import numpy as np
import pandas as pd
from datasets import Dataset, concatenate_datasets, load_from_disk
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import (
    AutoConfig,
    AutoModelForQuestionAnswering,
    AutoTokenizer,
    DataCollatorWithPadding,
    EvalPrediction,
    HfArgumentParser,
    TrainingArguments,
    set_seed,
)

In [219]:
data_path = "../../data"
context_path = "wikipedia_documents.json"
with open(os.path.join(data_path, context_path), "r", encoding="utf-8") as f:
    wiki = json.load(f)

contexts = list(
    dict.fromkeys([v["text"] for v in wiki.values()])
)

In [220]:
need_del = [60,270,344,586,629,630,644,655, 973, 4529,4525,4526,4527,4528, 4530,4531,4532,4533,4534,29028,31111,37157]
for i in need_del:
    contexts[i] = ""

In [221]:
a = pd.read_csv("./checkcheck.csv")
a = list(a['question'])
query = a

In [5]:
for i, v in enumerate(query):
    for ii in range(len(v)-1,0,-1):
        if query[i][ii] == " ":
            query[i] = query[i][0:ii]
            break

In [7]:
for i in range(len(query)):
    query[i] = query[i][0:-1]

In [222]:
tokenizer = AutoTokenizer.from_pretrained(
        "monologg/kobigbird-bert-base",
        use_fast=True,
    )

    

tfidf = TfidfVectorizer(
            tokenizer=tokenizer.tokenize, ngram_range=(1, 2), max_features=10000000,
        )

In [11]:
context_vec = tfidf.fit_transform(contexts)
query_vec = tfidf.transform(query)

Token indices sequence length is longer than the specified maximum sequence length for this model (1133 > 512). Running this sequence through the model will result in indexing errors


In [223]:
context_vecc = tfidf.fit_transform(contexts)
query_vecc = tfidf.transform(query)

Token indices sequence length is longer than the specified maximum sequence length for this model (9609 > 4096). Running this sequence through the model will result in indexing errors


In [224]:
a = pd.read_csv("./checkcheck.csv")
a= list(a['original_context'])
tmp = []
for v in a:
    for ii,vv in enumerate(contexts):
        if v == vv:
            tmp.append(ii)

In [25]:
    result = query_vec * context_vec.T
    #
    for k in range(1, 20):
        doc_scores = []
        doc_indices = []
        if not isinstance(result, np.ndarray):
            result = result.toarray()
        for i in range(result.shape[0]):
            sorted_result = np.argsort(result[i, :])[::-1]
            doc_scores.append(result[i, :][sorted_result].tolist()[:k])
            doc_indices.append(sorted_result.tolist()[:k])
        ccnt = 0
        indice = 0
        index = []
        for i, ii in zip(tmp, doc_indices):
            if i in ii:
             ccnt+=1
            else:
             index.append(indice)
            indice+=1
        print("top ",k," acc: ", ccnt/240*100)

top  1  acc:  45.0
top  2  acc:  64.16666666666667
top  3  acc:  73.75
top  4  acc:  77.91666666666667
top  5  acc:  79.16666666666666
top  6  acc:  81.25
top  7  acc:  82.91666666666667
top  8  acc:  82.91666666666667
top  9  acc:  83.33333333333334
top  10  acc:  85.0
top  11  acc:  85.41666666666666
top  12  acc:  86.66666666666667
top  13  acc:  86.66666666666667
top  14  acc:  86.66666666666667
top  15  acc:  87.08333333333333
top  16  acc:  87.08333333333333
top  17  acc:  87.08333333333333
top  18  acc:  87.5
top  19  acc:  87.5


In [235]:
        result = query_vecc * context_vecc.T
        k = 5
    #for k in range(5):
        doc_scores = []
        doc_indices = []
        if not isinstance(result, np.ndarray):
            result = result.toarray()
        for i in range(result.shape[0]):
            sorted_result = np.argsort(result[i, :])[::-1]
            doc_scores.append(result[i, :][sorted_result].tolist()[:k])
            doc_indices.append(sorted_result.tolist()[:k])
        ccnt = 0
        indice = 0
        index = []
        for i, ii, iii in zip(tmp, doc_indices, ttmp):
            for iiii in ii:
                if iii in contexts[iiii] or i in ii:
                    ccnt+=1
                else:
                    index.append(indice)
                break
            indice+=1
        print("top ",k," acc: ", ccnt/240*100)
        print(ccnt)

top  5  acc:  78.75
189


In [236]:
index

[5,
 8,
 19,
 20,
 21,
 22,
 26,
 55,
 63,
 65,
 69,
 74,
 75,
 80,
 98,
 101,
 107,
 111,
 118,
 122,
 123,
 125,
 128,
 130,
 131,
 132,
 135,
 142,
 148,
 153,
 160,
 161,
 162,
 163,
 168,
 169,
 170,
 171,
 180,
 186,
 204,
 207,
 209,
 213,
 216,
 221,
 224,
 226,
 230,
 232,
 236]

In [20]:
top  0  acc:  0.0
0
top  5  acc:  79.16666666666666
190
top  10  acc:  85.41666666666666
205
top  15  acc:  87.08333333333333
209
top  20  acc:  88.33333333333333
212
top  25  acc:  88.75
213
top  30  acc:  89.58333333333334
215
top  35  acc:  90.41666666666667
217
top  40  acc:  91.66666666666666
220
top  45  acc:  92.91666666666667
223
top  50  acc:  93.75
225
top  55  acc:  95.0
228
top  60  acc:  95.0
228
top  65  acc:  95.41666666666667
229
top  70  acc:  95.41666666666667
229
top  75  acc:  96.25
231
top  80  acc:  96.25
231
top  85  acc:  96.25
231
top  90  acc:  96.25
231
top  95  acc:  96.25

['처음으로 부실 경영인에 대한 보상 선고를 받',
 '스카버러 남쪽과 코보콘그 마을의 철도 노선이 처음 연장',
 '촌락에서 운영 위원 후보자 이름을 쓰기위해 사용',
 '로타이르가 백조를 구하기 위해 사용',
 '의견을 자유롭게 나누는 것은 조직 내 어떤 관계에',
 '1945년 쇼와천황의 항복 선언이 발표된 라디',
 '징금수는 서양 자수의 어떤 기법과 같은 기술',
 '다른 과 의사들은 감염내과 전문의들로부터 어떤 것에 대해 조언',
 '루이 14세의 왕비 마리아 테래사는 어느 나',
 '헤자즈 왕국이 실존했던 것',
 '버드 교장이 5월의 여왕의 대안으로 제시',
 "인형사'를 만들어낸 것으로 추측되는 사업",
 '멘데스가 요원들을 구하기 위해 간 도시',
 '교과부의 행동에 화가나 여러명이 사직한 기구',
 '반대동맹이 공산당과 갈라서겠다고 얘기한 날',
 '피어슨이 다시 의회를 해산했',
 '몽케가 죽은 뒤 쿠릴타이에서 대칸의 지위를 얻은 사람',
 '이흥구의 사법시험 이야기를 기사로 작성',
 '남북조 시대에서 이이 씨가 전쟁이 발생했을 때, 생활했',
 '박지훈은 1라운드에서 몇 순위',
 '데메카론에는 무엇을 풍자하는 이야기',
 '병에 걸려 죽을 확률이 약 25~50%에 달하는 유형',
 '설리반이 불만을 표시한 대상',
 '베소스는 어디',
 '진전사의 명칭이 드러나는 데 영향을 ',
 '자신의 이상적인 국가관이 스파르타와 닮아 있다고 생각하는 플라톤',
 '박제된 북극곰이 사망',
 '문법 측면에서 더 보수적인 포르투갈',
 '로스 수장이 살해한 사람은 어느 ',
 '조경숙왕의 아들인 요자의 친어머니',
 '오래플린과 부스의 마지막 계획에 따르면 그들은 어디서 링컨을 납치하려',
 '김득황이 친일파로 취급되었던 것은 무슨 경',
 '레닌이 출간한 책 중 농민의 자발적 참여에 대한 내용이 포함되어있',
 '신란의 동반자가 죽었다고 전해지',
 '칭자오의 머리가 엄청난 위력을 발휘할 수 없게 된 것은 누',
 '해초나 조류 표면에서 자라는 

In [31]:
ans = pd.read_csv("./check2.csv")

In [192]:
ans['answers'][11]

'{\'answer_start\': [901], \'text\': ["\'일급 비밀 프로젝트 2501\'"]}'

In [184]:
import re

re.findall(r"ext\'\:\s\[(.+)\]\}", ans['answers'][2])[0]


"'나뭇잎'"

In [216]:
ttmp=[]
for i in range(240):
    a = re.findall(r"ext\'\:\s\[(.+)\]\}", ans['answers'][i])
    a = a[0]
    a = a.replace("'", "")
    a = a.replace("'", "")
    ttmp.append(a)

밥


In [217]:
ttmp

['한보철강',
 '1871년',
 '나뭇잎',
 '금대야',
 '수평적 관계',
 '옥음방송',
 '코칭 스티치',
 '복잡한 감염병',
 '스페인',
 '20세기 초',
 '"5월의 왕"',
 '"일급 비밀 프로젝트 2501"',
 '테헤란',
 '역사교육과정개발추진위원회',
 '1967년 11월 15일',
 '1965년',
 '아리크 부케',
 '〈중앙일보〉',
 '미타케성',
 '전체 4순위',
 '뇌물',
 '보통 유형 준융합성 천연두',
 '데스탱 장군',
 '박트리아',
 '"진전(陳田)이라 새겨진 기와조각"',
 '《국가》',
 '2011년 3월 19일',
 '유럽 포르투갈어',
 '메이저 릿지',
 '요희',
 '포드 극장',
 '만주국 관리',
 '『협동조합에 관하여』',
 '도고쿠',
 '몽키 D. 가프',
 '조류',
 '광주교도소',
 '작가 베게티우스',
 '가오슝 시',
 '쇠망치',
 '마리즈 교수',
 '런던',
 '이틀',
 '의상대사',
 '홋카이도',
 '‘우유의 바다’',
 '장어',
 '소련',
 '전치',
 '흑색육(黑色肉)',
 '욱',
 '독일인민당',
 '하기노 역',
 '회칠',
 '황강다리',
 '직업 교육',
 '공산당',
 '매년 음력 정월',
 '조계종',
 '베이징',
 '독일군',
 '1932년',
 '제서지전(齊西之戰)이후',
 '가루아',
 '출생 천궁도',
 '나말여초',
 '1939년',
 '창 절제술',
 '도버 밀',
 '여정현(呂正鉉)',
 '광배',
 '울산',
 '땅(은색)과 하늘(금색)을 맺고 푸는 권한',
 '저수지',
 '퀘이커 교',
 '피에르 오주로',
 '지분',
 '독일',
 '제 3자',
 '페이팔',
 '가리타 히사노리',
 '점수',
 '코스모케라톱스',
 '"사채회사"',
 '스위스',
 '"공유지의 비극"',
 '그 자체가 계급 모순을 가지는 체제',
 '해리 트루먼 대통령',
 '무위태수',
 '크라운라이터',
 '1990년 9월',
 '"인간과 요괴의 완전한 평

11    {'answer_start': [901], 'text': ["'일급 비밀 프로젝트 ...
Name: answers, dtype: object